In [1]:
## Data Transformation - FIPS 
## Version 7
### 1/12/25
### 21:25

In [2]:
import sys
import os

# Add the root directory to sys.path
sys.path.append(os.path.abspath(".."))

# Now you can use absolute imports
from d497_helpers import folder_manager as fm, archive_module, checkpoint_helper as CheckPoint, database_helper as db_tool, config

# Pandas
import pandas as pd

# Numpy
import numpy as np

# RegularExpression for string matching
import re

# GLOB 
import glob 

# Pathlib 
from pathlib import Path



In [3]:
fips_crossreference_df = pd.DataFrame()

In [4]:
# Initializer Commands

# Initializes Database Engine and Session For SQL
db_tool.initialize_engine()
db_tool.initialize_session()


In [5]:
sql_query = "SELECT state_name AS state_lookup, state_fipcode AS fips FROM fips_data UNION ALL SELECT state_code AS state_lookup, state_fipcode AS fips FROM fips_data"

In [6]:
fips_crossreference_df = db_tool.export_df_from_sql(sql_query, fips_crossreference_df)

2025-01-20 17:52:45,884 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-20 17:52:45,886 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT state_name AS state_lookup, state_fipcode AS fips FROM fips_data UNION ALL SELECT state_code AS state_lookup, state_fipcode AS fips FROM fips_data")
2025-01-20 17:52:45,887 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-20 17:52:45,889 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT state_name AS state_lookup, state_fipcode AS fips FROM fips_data UNION ALL SELECT state_code AS state_lookup, state_fipcode AS fips FROM fips_data")
2025-01-20 17:52:45,890 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-20 17:52:45,891 INFO sqlalchemy.engine.Engine SELECT state_name AS state_lookup, state_fipcode AS fips FROM fips_data UNION ALL SELECT state_code AS state_lookup, state_fipcode AS fips FROM fips_data
2025-01-20 17:52:45,892 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-20 17:52:46,021 INFO sqlalchemy.engine.Engin

In [7]:
fips_crossreference_df

,state_lookup,fips
0,ALABAMA,01
1,ALABAMA,01
2,ALABAMA,01
3,ALABAMA,01
4,ALABAMA,01
...,...,...
79577,WY,56
79578,WY,56
79579,WY,56
79580,WY,56


In [8]:
fips_crossreference_df.drop_duplicates(inplace=True)

In [9]:
fips_crossreference_df.sort_values(by="fips", inplace=True)

In [10]:
fips_crossreference_df.reset_index()

,index,state_lookup,fips
0,0,ALABAMA,01
1,39791,AL,01
2,801,ALASKA,02
3,40592,AK,02
4,1333,ARIZONA,04
...,...,...,...
97,37505,WEST VIRGINIA,54
98,78419,WI,55
99,38628,WISCONSIN,55
100,39569,WYOMING,56


In [11]:
fips_crossreference_df.set_index('state_lookup')

,fips
state_lookup,
ALABAMA,01
AL,01
ALASKA,02
AK,02
ARIZONA,04
...,...
WEST VIRGINIA,54
WI,55
WISCONSIN,55


In [12]:
fips_crossreference_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102 entries, 0 to 79360
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   state_lookup  102 non-null    object
 1   fips          102 non-null    object
dtypes: object(2)
memory usage: 2.4+ KB


In [13]:
db_tool.export_to_sql(fips_crossreference_df, "state_lookup")

2025-01-20 17:52:46,106 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-20 17:52:46,109 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("state_lookup")
2025-01-20 17:52:46,111 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-20 17:52:46,113 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("state_lookup")
2025-01-20 17:52:46,114 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-20 17:52:46,115 INFO sqlalchemy.engine.Engine 
CREATE TABLE state_lookup (
	state_lookup TEXT, 
	fips TEXT
)


2025-01-20 17:52:46,117 INFO sqlalchemy.engine.Engine [no key 0.00107s] ()
2025-01-20 17:52:46,124 INFO sqlalchemy.engine.Engine INSERT INTO state_lookup (state_lookup, fips) VALUES (?, ?)
2025-01-20 17:52:46,125 INFO sqlalchemy.engine.Engine [generated in 0.00132s] [('ALABAMA', '01'), ('AL', '01'), ('ALASKA', '02'), ('AK', '02'), ('ARIZONA', '04'), ('AZ', '04'), ('ARKANSAS', '05'), ('AR', '05')  ... displaying 10 of 102 total bound parameter sets ...  ('WYOMING', '56'), ('WY', '56')]


In [14]:
#fips_crossreference_file_name_csv = "state_lookup.csv"
#fips_crossreference_file_name_pickle = "state_lookup.pkl"

#fips_crossreference_file_path_csv = config.processed_fips_data_folder_path + "/" + fips_crossreference_file_name_csv
#fips_crossreference_file_path_pickle = config.processed_fips_data_folder_path + "/" + fips_crossreference_file_name_pickle

#fips_crossreference_df.to_csv(fips_crossreference_file_path_csv, index=False)
#fips_crossreference_df.to_pickle(fipes_crossreference_file_path_pickle)

In [15]:
CheckPoint.create_checkpoint("cleaned_fips_data", "state_lookup", fips_crossreference_df)

In [16]:
commit_test_sql_query = "SELECT * FROM state_lookup"

In [17]:
commit_test_fips_crossreference_df = pd.DataFrame()

In [18]:
commit_test_fips_crossreference_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


In [19]:
commit_test_fips_crossreference_df = db_tool.export_df_from_sql(commit_test_sql_query, commit_test_fips_crossreference_df)

2025-01-20 17:52:46,188 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-20 17:52:46,189 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM state_lookup")
2025-01-20 17:52:46,190 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-20 17:52:46,191 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM state_lookup")
2025-01-20 17:52:46,192 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-20 17:52:46,193 INFO sqlalchemy.engine.Engine SELECT * FROM state_lookup
2025-01-20 17:52:46,194 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-20 17:52:46,196 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
commit_test_fips_crossreference_df

,state_lookup,fips
0,ALABAMA,01
1,AL,01
2,ALASKA,02
3,AK,02
4,ARIZONA,04
...,...,...
97,WEST VIRGINIA,54
98,WI,55
99,WISCONSIN,55
100,WYOMING,56


In [21]:
del commit_test_fips_crossreference_df